In [2]:
import os
import pandas as pd
import cv2
import numpy as np
from random import shuffle
import pickle

In [3]:
train_dir = 'train_data'
df = pd.read_csv('train_labels.csv')
IMG_SIZE = 28
LR = 1e-3

In [12]:
def label_image(img):
    result = df.loc[df['id']==int(img)]['label']
    if result.tolist()[0] == 'cat':
        return 0
    else:
        return 1

In [5]:
def preprocess_training_data():
    training_data = []
    for img_path in os.listdir(train_dir):
        label = label_image(img_path.split('.')[0])
        image = cv2.resize(cv2.imread(os.path.join(train_dir,img_path),cv2.IMREAD_UNCHANGED),(IMG_SIZE,IMG_SIZE))
        training_data.append([image,label])
    shuffle(training_data)
    return training_data

In [13]:
training_data = preprocess_training_data()

In [14]:
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X)/255
y = np.array(y)


In [11]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [16]:
model = Sequential()

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128,input_shape=X.shape[1:],activation='relu'))

model.add(Dense(2, activation='softmax'))

In [21]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(X,y,epochs=10,validation_split=0.1)

Epoch 1/10
225/225 [==============================] - 9s 38ms/step - loss: 0.6758 - accuracy: 0.5799 - val_loss: 0.6602 - val_accuracy: 0.6050
Epoch 2/10
225/225 [==============================] - 8s 36ms/step - loss: 0.6381 - accuracy: 0.6404 - val_loss: 0.6483 - val_accuracy: 0.5987
Epoch 3/10
225/225 [==============================] - 8s 36ms/step - loss: 0.6063 - accuracy: 0.6668 - val_loss: 0.6142 - val_accuracy: 0.6500
Epoch 4/10
225/225 [==============================] - 8s 36ms/step - loss: 0.5845 - accuracy: 0.6904 - val_loss: 0.6149 - val_accuracy: 0.6475
Epoch 5/10
225/225 [==============================] - 8s 35ms/step - loss: 0.5557 - accuracy: 0.7090 - val_loss: 0.5805 - val_accuracy: 0.6850
Epoch 6/10
225/225 [==============================] - 8s 37ms/step - loss: 0.5324 - accuracy: 0.7267 - val_loss: 0.6772 - val_accuracy: 0.6612
Epoch 7/10
225/225 [==============================] - 8s 37ms/step - loss: 0.5011 - accuracy: 0.7528 - val_loss: 0.5940 - val_accuracy: 0.6612

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (32, 26, 26, 64)          1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (32, 13, 13, 64)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (32, 11, 11, 64)          36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (32, 5, 5, 64)           0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (32, 1600)                0         
                                                                 
 dense_2 (Dense)             (32, 128)                